In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
from UNeuralNetwork import LinearLayer, LayerSequential

In [2]:
#Реклама
data = pd.read_csv('DATA/Advertising.csv')
X = data.drop('sales', axis=1)
y = data['sales']

In [2]:
class Perceptron(torch.nn.Module):
    def __init__(self,
                 input_dim,
                 hidden_dim,
                 output_dim,
                 num_layers):
        super(Perceptron, self).__init__()
        self.layers = torch.nn.Sequential()
        cur_dim = input_dim
        for index in range(num_layers):
            self.layers.add_module(f'Linear{index+1}', torch.nn.Linear(cur_dim, hidden_dim, dtype=torch.float64))
            self.layers.add_module(f'Relu{index+1}', torch.nn.ReLU())
            cur_dim = hidden_dim
        self.layers.add_module(f'Regression', torch.nn.Linear(cur_dim, output_dim, dtype=torch.float64))

    def forward(self, x):
        return self.layers(x)

def trainer(model, 
            X_train, 
            y_train,
            optimizer,
            loss_function,
            epochs=10):
    model.train()
    for _ in range(epochs):
        model.zero_grad()
        output = model(X_train).reshape(-1,)
        loss = loss_function(output, y_train)
        loss.backward()
        optimizer.step()

def tester(model,
           X_test,
           y_test):
    model.eval()
    with torch.no_grad():
        y_predict = model(X_test).reshape(-1,)
        print(y_predict[:5].numpy())
        print(y_test[:5].numpy())
        print(f'Ошибка на тестовой выборке = {root_mean_squared_error(y_test, y_predict)}')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.from_numpy(X_train).to('cpu')
X_test = torch.from_numpy(X_test).to('cpu')
y_train = torch.from_numpy(y_train.values).to('cpu')
y_test = torch.from_numpy(y_test.values).to('cpu')

In [5]:
model = Perceptron(3,5,1,2).to('cpu')

trainer(model, 
        X_train,
        y_train,
        torch.optim.Adam(model.parameters()),
        torch.nn.MSELoss(),
        2000)

tester(model,
       X_test,
       y_test)

[14.24614743 19.1124509  11.53712033 16.18959438  8.54227422]
[14.7 19.8 11.9 16.7  9.5]
Ошибка на тестовой выборке = 0.7105478354395045


In [6]:
model

Perceptron(
  (layers): Sequential(
    (Linear1): Linear(in_features=3, out_features=5, bias=True)
    (Relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (Relu2): ReLU()
    (Regression): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [104]:
class USequential:
    def __init__(self):
        self.layers_name = []
        self.layers = []

    def __repr__(self):
        return ''.join([f'({layer_name}): {layer}\n'
                        for layer_name, layer in zip(self.layers_name, self.layers)])

    def __getitem__(self, index):
        return self.layers[index]
    
    def add_module(self, layer_name, layer):
        self.layers_name.append(layer_name)
        self.layers.append(layer)

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, x, y, loss_function_type):
        self.layers[-1].setErrors(self._loss_function[loss_function_type](x, y))
        further_layer = self.layers[-1]
        for layer in self.layers[-2::-1]:
            layer.backward(further_layer)
            further_layer = layer    

    def step(self, lr):
        for layer in self.layers:
            layer.step(lr)
        
    @staticmethod
    def _dirSquareError(x, y):
        vector = 2 * (x - y)
        return vector / vector.numel()

    _loss_function = {'MSE': _dirSquareError}


class ULinear:
    def __init__(self, 
                 in_features,
                 out_features,
                 bias=True,
                 start_weights_type='random'):
        self.in_features = in_features
        self.out_features = out_features
        self.weights_ = self._start_weights[start_weights_type]((in_features, out_features))
        self.bias_ = self._start_weights[start_weights_type]((out_features,)) if bias else torch.zeros((out_features,))

    def __repr__(self):
        return f'Linear(in_features={self.in_features}, out_features={self.out_features})'

    def forward(self, x):
        self.prev_output_ = x.detach().clone()
        return self.weights_ @ x + self.bias_

    def backward(self, further_layer):
        self.errors_ = further_layer._getPrevLayerErrors()

    def step(self, lr):
        self.weights_ -= lr * (self.errors_.reshape(-1,1) @ self.prev_output_.reshape(1,-1))

    def _getPrevLayerErrors(self):
        return self.errors_ @ self.weights_

    def setErrors(self, errors):
        self.errors_ = errors

    _start_weights = {'random': torch.randn}


class UReLU:
    def __init__(self):
        pass

    def __repr__(self):
        return 'ReLU()'

    def forward(self, x):
        self.z_ = torch.where(x >= 0, 1, 0)
        return torch.where(x >= 0, x, 0)

    def backward(self, further_layer):
        self.errors_ = further_layer._getPrevLayerErrors()

    def step(self, lr):
        pass

    def _getPrevLayerErrors(self):
        return self.errors_ * self.z_

    def setErrors(self, errors):
        self.errors_ = errors

class UTanh:
    def __init__(self):
        pass

    def __repr__(self):
        return 'Tanh()'

    def forward(self, x):
        self.z_ = -torch.pow(torch.tanh(x), 2) + 1
        return torch.tanh(x)

    def backward(self, further_layer):
        self.errors_ = further_layer._getPrevLayerErrors()

    def step(self, lr):
        pass

    def _getPrevLayerErrors(self):
        return self.errors_ * self.z_

    def setErrors(self, errors):
        self.errors_ = errors

In [105]:
class UConv2d:
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 bias=True,
                 start_weights_type='random'):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.weights_ = self._start_weights[start_weights_type]((out_channels, in_channels, kernel_size, kernel_size))
        self.bias_ = self._start_weights[start_weights_type]((out_channels,)) if bias else torch.zeros((out_channels,))

    def __repr__(self):
        return (f'Conv2d(in_channels={self.in_channels}, '
                       f'out_channels={self.out_channels}, '
                       f'kernel_size=({self.kernel_size},{self.kernel_size})')

    @staticmethod
    def _makeConvolution(X_channel, kernel):
        k_n, k_m = kernel.shape
        n = X_channel.shape[0] - k_n + 1
        m = X_channel.shape[1] - k_m + 1
        return torch.tensor([[torch.sum(X_channel[row:row+k_n, col:col+k_m]*kernel) 
                              for col in range(m)]
                             for row in range(n)]).reshape(1,n,m)

    def _getInputErrors(self, output_errors, kernel):
        reversed_kernel = torch.flip(kernel, (0,1))
        k_n, k_m = np.array(reversed_kernel.shape) - 1
        n, m = output_errors.shape
        extended_errors = torch.zeros((n + 2 * k_n, m + 2 * k_m))
        extended_errors[k_n:-k_n, k_m:-k_m] = output_errors
        return self._makeConvolution(extended_errors, reversed_kernel)

    def _getPrevLayerErrors(self):
        return torch.sum(torch.cat([torch.unsqueeze(torch.cat([self._getInputErrors(channel_error, kernel) 
                                                               for kernel in out_kernels], 
                                                              dim=0),
                                                    0)
                                    for out_kernels, channel_error in zip(self.weights_, self.errors_)]),
                         dim=0)

    def setErrors(self, errors):
        self.errors_ = errors
    
    def forward(self, X):
        self.prev_img_ = X.detach().clone()
        return torch.cat([torch.unsqueeze(torch.sum(torch.cat([self._makeConvolution(X_channel, kernel)
                                                               for X_channel, kernel in zip(X, out_kernels)]),
                                                    dim=-3), 
                                          0)
                          for out_kernels in self.weights_])

    def backward(self, further_layer):
        self.errors_ = further_layer._getPrevLayerErrors()

    def step(self, lr):
        weights_grad = torch.cat([torch.unsqueeze(torch.cat([c._makeConvolution(X_channel, channel_errors)
                                                             for X_channel in self.prev_img_]), 
                                                  0)
                                  for channel_errors in self.errors_])
        self.weights_ -= lr * weights_grad

    _start_weights = {'random': torch.randn}


class UFlatten:
    def __init__(self):
        pass

    def __repr__(self):
        return 'Flatten()'

    def forward(self, x):
        self.initial_shape = x.shape
        return x.reshape(self.initial_shape[0],-1)

    def backward(self, further_layer):
        self.errors_ = further_layer.errors_.reshape(self.initial_shape)

    def step(self, lr):
        pass

    def _getPrevLayerErrors(self):
        return self.errors_

    def setErrors(self, errors):
        self.errors_ = errors.reshape(self.initial_shape)

In [110]:
X_img = torch.tensor([[[2,3,4,1,5,3,2],
                       [6,7,3,4,6,3,2],
                       [1,7,3,6,5,4,8],
                       [6,3,5,0,9,6,5],
                       [4,5,2,8,6,3,2]],
                     
                      [[2,3,4,1,5,3,2],
                       [6,7,3,4,6,3,2],
                       [1,7,3,6,5,4,8],
                       [6,3,5,0,9,6,5],
                       [4,5,2,8,6,3,2]]],
                     dtype=torch.float32)
y_img = torch.tensor([[3,4,2,3,1,2,5,4],
                      [4,6,5,3,4,5,6,7],
                      [1,8,3,2,9,0,7,5],
                      [4,5,3,7,5,4,8,6]], dtype=torch.float32)

In [124]:
conv1 = torch.nn.Conv2d(2,3,3,bias=False)
conv2 = torch.nn.Conv2d(3,4,2,bias=False)

uconv1 = UConv2d(2,3,3,bias=False)
uconv1.weights_ = conv1.weight.detach().clone()
uconv2 = UConv2d(3,4,2,bias=False)
uconv2.weights_ = conv2.weight.detach().clone()

In [125]:
seq_img = torch.nn.Sequential()
seq_img.add_module('conv1', conv1)
seq_img.add_module('conv2', conv2)
seq_img.add_module('flatten', torch.nn.Flatten())

print(f'Изначальные параметры\nВеса\n{seq_img[0].weight.data}')

loss_function_img = torch.nn.MSELoss()
optimizer_img = torch.optim.SGD(seq_img.parameters(), lr=0.1)

for _ in range(2):
    optimizer_img.zero_grad()
    output_img = seq_img(X_img)
    loss_img = loss_function_img(output_img, y_img)
    loss_img.backward()
    optimizer_img.step()

print(f'Обновленные параметры\nВеса\n{seq_img[0].weight.data}')

Изначальные параметры
Веса
tensor([[[[-0.1807, -0.1847, -0.0076],
          [ 0.2078,  0.1143, -0.2108],
          [-0.1068, -0.1358, -0.2005]],

         [[ 0.1739, -0.1954,  0.1375],
          [ 0.2345, -0.1041,  0.1874],
          [-0.1819,  0.2081,  0.2306]]],


        [[[-0.0829,  0.2328,  0.0272],
          [-0.2142,  0.1262, -0.1044],
          [ 0.0636, -0.1200,  0.0448]],

         [[ 0.1689,  0.0575,  0.1964],
          [ 0.1837,  0.2242,  0.0451],
          [-0.0864, -0.0614, -0.1197]]],


        [[[ 0.0722,  0.2158,  0.0930],
          [ 0.2016,  0.2300,  0.0065],
          [ 0.2292,  0.1406,  0.1186]],

         [[ 0.2070,  0.2350,  0.0149],
          [-0.1643, -0.0201,  0.0344],
          [-0.0347, -0.1765, -0.1762]]]])
Обновленные параметры
Веса
tensor([[[[  16.4318,   20.5637,   19.5545],
          [  13.7620,   22.6163,   16.7182],
          [  12.0723,   18.3444,   15.1953]],

         [[  16.7865,   20.5530,   19.6995],
          [  13.7887,   22.3978,   17.1165],


In [126]:
useq_img = USequential()
useq_img.add_module('conv1', uconv1)
useq_img.add_module('conv2', uconv2)
useq_img.add_module('flatten', UFlatten())

print(f'Изначальные параметры\nВеса\n{useq_img[0].weights_}')

for _ in range(2):
    uOutput = useq_img.forward(X_img)
    useq_img.backward(uOutput, y_img, 'MSE')
    useq_img.step(0.1)

print(f'Обновленные параметры\nВеса\n{useq_img[0].weights_.data}')

Изначальные параметры
Веса
tensor([[[[-0.1807, -0.1847, -0.0076],
          [ 0.2078,  0.1143, -0.2108],
          [-0.1068, -0.1358, -0.2005]],

         [[ 0.1739, -0.1954,  0.1375],
          [ 0.2345, -0.1041,  0.1874],
          [-0.1819,  0.2081,  0.2306]]],


        [[[-0.0829,  0.2328,  0.0272],
          [-0.2142,  0.1262, -0.1044],
          [ 0.0636, -0.1200,  0.0448]],

         [[ 0.1689,  0.0575,  0.1964],
          [ 0.1837,  0.2242,  0.0451],
          [-0.0864, -0.0614, -0.1197]]],


        [[[ 0.0722,  0.2158,  0.0930],
          [ 0.2016,  0.2300,  0.0065],
          [ 0.2292,  0.1406,  0.1186]],

         [[ 0.2070,  0.2350,  0.0149],
          [-0.1643, -0.0201,  0.0344],
          [-0.0347, -0.1765, -0.1762]]]])
Обновленные параметры
Веса
tensor([[[[  16.4318,   20.5637,   19.5545],
          [  13.7620,   22.6163,   16.7182],
          [  12.0723,   18.3444,   15.1953]],

         [[  16.7865,   20.5530,   19.6995],
          [  13.7887,   22.3978,   17.1165],
